In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import csv

# MediaPipe 손 모델 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# 새로운 제스처 라벨 정의
new_gesture_label = 10.000000  # 새로운 제스처 라벨 번호 (소수점 6자리로 저장)

# 이미지 폴더 경로 (모든 이미지 파일을 처리할 폴더 경로)
image_folder = r'C:\jun_vs\dataset\10'  # 경로에 맞춰 수정
csv_file = 'new_gesture_data.csv'  # 결과를 저장할 CSV 파일

# CSV 파일에 관절 각도와 라벨 저장
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    
    # 이미지 폴더 내 모든 이미지 파일 반복
    for img_name in os.listdir(image_folder):
        # 파일 경로 만들기
        img_path = os.path.join(image_folder, img_name)
        
        # 파일이 실제로 존재하는지 확인
        if not os.path.exists(img_path):
            print(f"File not found: {img_path}")
            continue
        
        # 이미지 파일 읽기
        img = cv2.imread(img_path)

        # 이미지가 제대로 불러와졌는지 확인
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # MediaPipe로 손 관절 분석
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        if result.multi_hand_landmarks:
            for res in result.multi_hand_landmarks:
                joint = np.zeros((21, 3))
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z]
                
                # 부모 관절과 자식 관절 간의 벡터 계산
                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # 부모 관절
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # 자식 관절
                v = v2 - v1
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                # 각도 계산
                angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
                angle = np.degrees(angle)
                
                # 각도 데이터를 소수점 6자리로 제한하고, 리스트로 변환
                angle = np.round(angle, 6).tolist()  # 소수점 6자리로 제한
                
                # 각도 데이터와 제스처 라벨 추가 (소수점 6자리로)
                angle.append(np.round(new_gesture_label, 6))  # 라벨도 소수점 6자리
                
                # CSV 파일에 데이터 기록
                writer.writerow(angle)

print(f"New gesture data saved to {csv_file}")

New gesture data saved to new_gesture_data.csv
